<a href="https://colab.research.google.com/github/hudson-dev/tflite-convert/blob/master/Checkpoint_to_TFLite_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing Dependencies**

Hudson, you may want to check that you install all these dependencies necessary for Tensorflow.

In [1]:
%cd /content
%rm -rf models
%tensorflow_version 1.14
!git clone --quiet https://github.com/edsumpena/models.git models

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools
!pip install tf_slim
!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Selecting previously unselected package python-bs4.
(Reading database ... 146425 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webenco

In [4]:
!ls -a
!pwd

.		    deeplab		   pcl_rl
..		    deep_speech		   README.md
a3c_blogpost	    delf		   rebar
adversarial_text    efficient-hrl	   sequence_projection
attention_ocr	    lfads		   setup.py
audioset	    lstm_object_detection  slim
autoaugment	    marco		   vid2depth
cognitive_planning  nst_blogpost
cvt_text	    object_detection
/content/models/research


## **Converting Checkpoint file to Tensorflow Lite Graph**

Hudson, this was the first mistake for Tensorflow Lite conversion. You needed to convert the .ckpt file to a .pb for Tensorflow Lite. I think you may have converted to an incorrect .pb file (although I am not sure). Somehow, I was just not able to use the .pb file that you sent me and convert it to .tflite.

**Inputs:**
- Upload your pipeline.config file to the "/content" folder
- Upload your model.ckpt (all 3 files) to the "/content" folder

In [5]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path='/content/pipeline.config' \
    --trained_checkpoint_prefix='/content/model.ckpt-1624' \
    --output_directory='/content/saved_model_tflite' \
    --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0215 03:49:00.210954 140497665386368 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0215 03:49:01.790563 140497665386368 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0215 03:49:01.825140 140497665386368 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0215 03:49:01.859236 140497665386368 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

**Outputs:**
- A tflite_graph.pb (Tensorflow Lite Graph) under "/content/saved_model_tflite"
- A tflite_graph.pbtxt under "/content/saved_model_tflite"

## **Converting Tensorflow Lite Graph to tflite**

I was able to convert to tflite using the command. We made (me included) a key mistake: the inference_type should be FLOAT, not INT8. Though, I am not sure why "tflite_convert --help" didn't work on your computer.

**Inputs:**
- A Tensorflow Lite Graph under "/content/saved_model_tflite" (from previous step)

In [6]:
!echo "CONVERTING frozen graph to quantized TF Lite file..."
!mkdir '/content/Final_TFLITE/'
!tflite_convert \
  --output_file='/content/Final_TFLITE/quantized.tflite' \
  --graph_def_file='/content/saved_model_tflite/tflite_graph.pb' \
  --inference_type=FLOAT \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

CONVERTING frozen graph to quantized TF Lite file...
2021-02-15 03:49:29.040050: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-02-15 03:49:29.050207: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-02-15 03:49:29.050258: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bb57c9c2ddcc): /proc/driver/nvidia/version does not exist
2021-02-15 03:49:29.060762: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-02-15 03:49:29.061011: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1844a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-02-15 03:49:29.061048: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


**Output:**
- A .tflite file under "/content/Final_TFLITE/" (Export this file to Android Studio FTC Repo to use!)